# Test de rectangularisation fruste

_Jérôme Lacaille_ (20/12/2024)

In [1]:
# Import minimal.

import os
import numpy as np
import pandas as pd
import tabata as tbt
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline



## 1. Renormalisation des données.

In [ ]:
# Chargement des données.
datadir = "../Data/cleaned/"
final_file = datadir + "AFL1EB_cleaned_final.h5"
norm_file = datadir + "AFL1EB_normed_alt.h5"
dsc = tbt.Opset(final_file)
dsc

OPSET '../Data/cleaned/AFL1EB_cleaned_final.h5' de 47 signaux.
        position courante : sigpos  = 0
        variable courante : colname = ALT[m]
        phase surlignée   : phase   = None

In [7]:
dsc.plot()

In [8]:
# Ajout d'une référence normalisée à utiliser pour la transformation.

dsn1 = tbt.Opset(norm_file).clean()
for df in dsc:
    alt = df["ALT[m]"]
    max_alt = np.max(alt)
    min_alt = np.min(alt)
    altn = (alt-min_alt)/(max_alt-min_alt)
    df["ALTN[-]"] = altn
    dsn1.put(df)
dsn1

OPSET '../Data/cleaned/AFL1EB_normed.h5' de 47 signaux.
        position courante : sigpos  = 46
        variable courante : colname = ALT[m]
        phase surlignée   : phase   = None

In [9]:
dsn1.plot()

In [21]:
df = dsn1.df
x = df["ALTN[-]"]
x.index

DatetimeIndex(['2013-02-08 04:30:00', '2013-02-08 04:30:01',
               '2013-02-08 04:30:02', '2013-02-08 04:30:03',
               '2013-02-08 04:30:04', '2013-02-08 04:30:05',
               '2013-02-08 04:30:06', '2013-02-08 04:30:07',
               '2013-02-08 04:30:08', '2013-02-08 04:30:09',
               ...
               '2013-02-08 06:59:05', '2013-02-08 06:59:06',
               '2013-02-08 06:59:07', '2013-02-08 06:59:08',
               '2013-02-08 06:59:09', '2013-02-08 06:59:10',
               '2013-02-08 06:59:11', '2013-02-08 06:59:12',
               '2013-02-08 06:59:13', '2013-02-08 06:59:14'],
              dtype='datetime64[ns]', name='record_24', length=8955, freq='S')

In [27]:
df.index.name

'record_24'

In [25]:
t0 = x.index[0].value
t1 = x.index[-1].value
t = pd.to_datetime(np.linspace(t0,t1,500))
y = np.interp(t,x.index,x.values)


array([5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 4.62224067e-04,
       5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 5.54446662e-04,
       5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 5.54446662e-04,
       5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 5.54446662e-04,
       5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 5.54446662e-04,
       5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 5.54446662e-04,
       5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 5.54446662e-04,
       6.02002395e-04, 5.54446662e-04, 5.54446662e-04, 5.54446662e-04,
       5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 5.54446662e-04,
       5.54446662e-04, 5.54446662e-04, 5.39779946e-04, 4.64446311e-04,
       5.54446662e-04, 5.54446662e-04, 5.54446662e-04, 4.43557330e-04,
       4.43557330e-04, 4.43557330e-04, 4.43557330e-04, 4.43557330e-04,
       3.32667997e-04, 2.21778665e-04, 6.65335995e-04, 6.65335995e-04,
       6.65335995e-04, 7.79114227e-04, 8.87114660e-04, 8.87114660e-04,
      

In [48]:
# Création d'une interpolation linéaire.

w = len(dsc.df.columns)
n = len(dsc)
N = 500
Y = []
Cols = []
for df in dsn1:
    x = df["ALTN[-]"]
    t0 = x.index[0].value
    t1 = x.index[-1].value
    t = pd.to_datetime(np.linspace(t0,t1,500))
    y = np.interp(t,x.index,x.values)
    Y.append(list(y))
    Cols.append(df.index.name)

In [49]:
Y

[[0.00046998002584890147,
  0.00046998002584890147,
  0.00046998002584890147,
  0.00046998002584890147,
  0.00040617012336975676,
  0.00035248501938667606,
  0.00046998002584890147,
  0.00046998002584890147,
  0.0005773502037363413,
  0.00046998002584890147,
  0.00046998002584890147,
  0.00046998002584890147,
  0.00046998002584890147,
  0.00046998002584890147,
  0.00035248501938667606,
  0.00035248501938667606,
  0.00035248501938667606,
  0.0004631516446951005,
  0.0002349900129244507,
  0.00037249919022441547,
  0.0002015545660909411,
  0.00035248501938667606,
  0.0002349900129244507,
  0.0002349900129244507,
  0.00011749500646222533,
  0.00011749500646222533,
  0.0002349900129244507,
  0.00035248501938667606,
  0.0005874750323111268,
  0.00035248501938667606,
  0.00035248501938667606,
  0.0002349900129244507,
  0.00035248501938667606,
  0.000442902017624251,
  0.00035248501938667606,
  0.00046998002584890147,
  0.00033929919962401593,
  0.00035248501938667606,
  0.0004699800258489014

In [50]:
YY = np.concatenate(list(Y),axis=1)
YY.shape

AxisError: axis 1 is out of bounds for array of dimension 1

In [51]:
pd.DataFrame(data=Y,columns=Cols)

ValueError: 47 columns passed, passed data had 500 columns

In [34]:
Y

[array([4.69980026e-04, 4.69980026e-04, 4.69980026e-04, 4.69980026e-04,
        4.06170123e-04, 3.52485019e-04, 4.69980026e-04, 4.69980026e-04,
        5.77350204e-04, 4.69980026e-04, 4.69980026e-04, 4.69980026e-04,
        4.69980026e-04, 4.69980026e-04, 3.52485019e-04, 3.52485019e-04,
        3.52485019e-04, 4.63151645e-04, 2.34990013e-04, 3.72499190e-04,
        2.01554566e-04, 3.52485019e-04, 2.34990013e-04, 2.34990013e-04,
        1.17495006e-04, 1.17495006e-04, 2.34990013e-04, 3.52485019e-04,
        5.87475032e-04, 3.52485019e-04, 3.52485019e-04, 2.34990013e-04,
        3.52485019e-04, 4.42902018e-04, 3.52485019e-04, 4.69980026e-04,
        3.39299200e-04, 3.52485019e-04, 4.69980026e-04, 5.87475032e-04,
        7.04970039e-04, 5.87475032e-04, 7.04970039e-04, 5.87475032e-04,
        7.04970039e-04, 7.04970039e-04, 3.52485019e-04, 3.09160200e-04,
        2.34990013e-04, 1.17495006e-04, 1.17495006e-04, 3.52485019e-04,
        1.24558844e-04, 3.52485019e-04, 3.52485019e-04, 4.699800

In [35]:
len(Y)

47

In [36]:
len(Cols)

47